Building classifier on own data

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPool2D, Flatten, Dropout
from keras.utils import plot_model
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = '../input/images/fruits/training_set/'
test_dir = '../input/images/fruits/test_set/'

Create image data generator objects

In [3]:
train_datagen = ImageDataGenerator(rescale=1/255, zoom_range=0.5, horizontal_flip=True, vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1/255, zoom_range=0.5, horizontal_flip=True, vertical_flip=True)

Read the images

In [ ]:
train = train_datagen.flow_from_directory(train_dir, 
                                          class_mode='binary',  # two classes so binary
                                          target_size=(64, 64)) # resize images to 64 x 64


test = test_datagen.flow_from_directory(test_dir)